In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

np.seterr(divide='ignore', invalid='ignore')


{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [19]:
digits = load_digits()

# Loading Data

X= digits.data
y= digits.target

# standardize
X = (X - X.mean())/X.std()

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=0)

In [20]:
X_train.shape, X_test.shape, y_train.shape , y_test.shape

((1437, 64), (360, 64), (1437,), (360,))

In [21]:
class SoftmaxRegression(object):

    def __init__(self, lr=0.01, epochs=50,
                 l2=0.01,
                 minibatches=1,
                 n_classes=None,
                 random_seed=None):

        self.lr = lr
        self.epochs = epochs
        self.l2 = l2
        self.minibatches = minibatches
        self.n_classes = n_classes
        self.random_seed = random_seed
        
    def _init_params(self, weights_shape, bias_shape=(1,), dtype='float64',
                     scale=0.01, random_seed=None):
        """Initialize weight coefficients."""
        if random_seed:
            np.random.seed(random_seed)
        w = np.random.normal(loc=0.0, scale=scale, size=weights_shape)
        b = np.zeros(shape=bias_shape)
        return b.astype(dtype), w.astype(dtype)

    def fit(self, X, y, init_params=True):

        if self.random_seed is not None:
            np.random.seed(self.random_seed)
        self._fit(X=X, y=y, init_params=init_params)
        self._is_fitted = True
        return self

    def _predict(self, X):
        probas = self.predict_proba(X)
        return self._to_classlabels(probas)

    def predict(self, X):

        if not self._is_fitted:
            raise AttributeError('Model is not fitted, yet.')
        return self._predict(X)

    def predict_proba(self, X):

        net = self.hx(X, self.weights, self.bias)
        pred = self._softmax(net)
        return pred

    def hx(self, X, W, b):
        return (X.dot(W) + b)

    def _softmax(self, z):
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

    def _cross_entropy(self, output, y_target):
        return - np.sum(np.log(output) * (y_target), axis=1)

    def _cost(self, cross_entropy):
        L2_term = self.l2 * np.sum(self.weights ** 2)
        cross_entropy = cross_entropy + L2_term
        return 0.5 * np.mean(cross_entropy)

    def _to_classlabels(self, z):
        return z.argmax(axis=1)

    def _one_hot(self, y, n_labels, dtype):

        mat = np.zeros((len(y), n_labels))
        for i, val in enumerate(y):
            mat[i, val] = 1
        return mat.astype(dtype)

    def _yield_minibatches_idx(self, n_batches, data_ary, shuffle=True):
        indices = np.arange(data_ary.shape[0])

        if shuffle:
            indices = np.random.permutation(indices)
        if n_batches > 1:
            remainder = data_ary.shape[0] % n_batches

            if remainder:
                minis = np.array_split(indices[:-remainder], n_batches)
                minis[-1] = np.concatenate((minis[-1],
                                            indices[-remainder:]),
                                           axis=0)
            else:
                minis = np.array_split(indices, n_batches)

        else:
            minis = (indices,)

        for idx_batch in minis:
            yield idx_batch

    def _shuffle_arrays(self, arrays):
        """Shuffle arrays in unison."""
        r = np.random.permutation(len(arrays[0]))
        return [ary[r] for ary in arrays]

    def _fit(self, X, y, init_params=True):
        if init_params:
            if self.n_classes is None:
                self.n_classes = np.max(y) + 1
            self._n_features = X.shape[1]

            self.bias, self.weights = self._init_params(
                weights_shape=(self._n_features, self.n_classes),
                bias_shape=(self.n_classes,),
                random_seed=self.random_seed)
            self.cost_ = []

        y_enc = self._one_hot(y=y, n_labels=self.n_classes, dtype=np.float)
        
        self.avg_weight=0
        self.avg_bias =0
        self.beta=0.9
        

        for i in range(self.epochs):
            for idx in self._yield_minibatches_idx(
                    n_batches=self.minibatches,
                    data_ary=y,
                    shuffle=True):

                z = self.hx(X[idx], self.weights, self.bias)
                pred = self._softmax(z)
                diff = pred - y_enc[idx]

                # gradient -> n_features x n_classes
                grad = np.dot(X[idx].T, diff)

                
                ## Momentum 
                self.avg_weight = self.avg_weight * self.beta + (1.0 - self.beta) * grad
                self.avg_bias = self.avg_bias * self.beta + (1.0- self.beta) * grad
                
                # update in opp. direction of the cost gradient
                self.weights -= (self.lr * self.avg_weight + self.lr * self.l2 * self.weights)
                self.bias -= (self.lr * np.sum(diff, axis=0))
                
                
                # decay learning rate
                self.lr = self.lr * 0.95

            # compute cost of the whole epoch
            z = self.hx(X, self.weights, self.bias)
            pred = self._softmax(z)
            cross_ent = self._cross_entropy(output=pred, y_target=y_enc)
            cost = self._cost(cross_ent)
        return self

In [22]:
# standardize
X_train.shape, y_train.shape

((1437, 64), (1437,))

## Gradient Descent

In [23]:
# Loading Data

X= digits.data
y= digits.target


# standardize
X = (X - X.mean())/X.std()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


multiClassLogReg = SoftmaxRegression(lr=0.01, epochs=10, minibatches=1, random_seed=0)
multiClassLogReg.fit(X_train, y_train)

y_pred = multiClassLogReg.predict(X_test)
print("model test accuracy is : ",  accuracy_score(y_test,y_pred) )

model test accuracy is :  0.9361111111111111


In [24]:
multiClassLogReg = SoftmaxRegression(lr=0.01, epochs=800, minibatches=1, random_seed=0)
multiClassLogReg.fit(X_train, y_train)
y_pred = multiClassLogReg.predict(X_test)
tr_y_pred = multiClassLogReg.predict(X_train)

In [25]:
print("=="*25)
print("Gradient Descent model test accuracy is : ",accuracy_score(y_test,y_pred )  )
print("=="*25)

Gradient Descent model test accuracy is :  0.9638888888888889


In [26]:
print("=="*25)
print("Gradient Descent model train accuracy is : ",accuracy_score(y_train,tr_y_pred)  )
print("=="*25)

Gradient Descent model train accuracy is :  0.9798190675017397


## Stochastic Gradient Descent

In [27]:
# Loading Data

X= digits.data
y= digits.target

# standardize
X = (X - X.mean())/X.std()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr = SoftmaxRegression(lr=0.05, epochs=200, minibatches=50, random_seed=0)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
tr_y_pred = multiClassLogReg.predict(X_train)

In [28]:
print("=="*25)
print("SGD model test accuracy is : ",accuracy_score(y_test,y_pred )  )
print("=="*25)

SGD model test accuracy is :  0.95


In [29]:
print("=="*25)
print("SGD model train accuracy is : ",accuracy_score(y_train,tr_y_pred)  )
print("=="*25)

SGD model train accuracy is :  0.9798190675017397


# SK Learn Implementation

In [30]:
from sklearn.linear_model import LogisticRegression

logRegres = LogisticRegression()

logRegres.fit(X_train, y_train)

sk_y_pred = logRegres.predict(X_test)

print("=="*25)
print("SK Learn Logistic Regression model test accuracy is : ",  accuracy_score(y_test,sk_y_pred) )
print("=="*25)

/home/pranay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/pranay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


SK Learn Logistic Regression model test accuracy is :  0.9611111111111111


In [31]:
sk_tr_y_pred = logRegres.predict(X_train)

print("=="*25)
print("SK Learn Logistic Regression model train accuracy is : ",  accuracy_score(y_train,sk_tr_y_pred) )
print("=="*25)

SK Learn Logistic Regression model train accuracy is :  0.988865692414753


## Result Compare

In [32]:
import pandas as pd
from prettytable import PrettyTable

x = PrettyTable()
x.field_names =  ['Metric','Sklearn','GD','SGD']
x.add_row(["Accuracy ", 0.96111,0.96388,0.95])


print('\n')
print(x)



+-----------+---------+---------+------+
|   Metric  | Sklearn |    GD   | SGD  |
+-----------+---------+---------+------+
| Accuracy  | 0.96111 | 0.96388 | 0.95 |
+-----------+---------+---------+------+


## Referernce :
- https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/softmax-regression.ipynb
- https://github.com/python-engineer/MLfromscratch/tree/master/mlfromscratch
- https://engmrk.com/gradient-descent-with-momentum/

In [33]:
## Mathematical terms:

# https://datascience.stackexchange.com/questions/20296/cross-entropy-loss-explanation